<a href="https://colab.research.google.com/github/fjme95/python-para-la-ciencia-de-datos/blob/main/semana%202/Log%C3%ADstica_Clasificaci%C3%B3n_de_SPAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivo

# Dependencias

In [1]:
!pip install -U pandas scikit-learn

     |████████████████████████████████| 11.3 MB 5.2 MB/s 
     |████████████████████████████████| 23.2 MB 1.7 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.4 which is incompatible.


In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

import plotly.express as px

In [2]:
nltk.download('stopwords')
nltk.download('punkt')

stemmer = PorterStemmer()
cached_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Datos

Datos obtenidos de https://www.kaggle.com/uciml/sms-spam-collection-dataset?select=spam.csv

In [3]:
!head spam.csv

v1,v2,,,
ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",,,
ham,Ok lar... Joking wif u oni...,,,
spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,,,
ham,U dun say so early hor... U c already then say...,,,
ham,"Nah I don't think he goes to usf, he lives around here though",,,
spam,"FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, �1.50 to rcv",,,
ham,Even my brother is not like to speak with me. They treat me like aids patent.,,,
ham,As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune,,,
spam,WINNER!! As a valued network customer you have been selected to receivea �900 prize reward! To claim call 09061701461. Claim 

## Lectura y exploración

In [4]:
data = pd.read_csv('spam.csv', usecols = ['v1', 'v2'], encoding_errors='ignore')
data

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will _ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


¿Cuántos valores tenemos de cada clase?

In [5]:
# Código para contar cuantos sms tenemos como spam y cuántos como ham

In [6]:
for txt in data[data.v1 == "ham"].sample(5).v2:
    print(txt, end = '\n\n')

If I get there before you after your ten billion calls and texts so help me god

Yo, you gonna still be in stock tomorrow/today? I'm trying to get a dubsack

Did he say how fantastic I am by any chance, or anything need a bigger life lift as losing the will 2 live, do you think I would be the first person 2 die from N V Q? 

Good morning. At the repair shop--the ONLY reason i'm up at this hour.

Wait.i will come out.. &lt;#&gt;  min:)



In [7]:
for txt in data[data.v1 == "spam"].sample(5).v2:
    print(txt, end = '\n\n')

I don't know u and u don't know me. Send CHAT to 86688 now and let's find each other! Only 150p/Msg rcvd. HG/Suite342/2Lands/Row/W1J6HL LDN. 18 years or over.

SMS AUCTION - A BRAND NEW Nokia 7250 is up 4 auction today! Auction is FREE 2 join & take part! Txt NOKIA to 86021 now!

Do you want a New Nokia 3510i colour phone DeliveredTomorrow? With 300 free minutes to any mobile + 100 free texts + Free Camcorder reply or call 08000930705

Your credits have been topped up for http://www.bubbletext.com Your renewal Pin is tgxxrz

No 1 POLYPHONIC tone 4 ur mob every week! Just txt PT2 to 87575. 1st Tone FREE ! so get txtin now and tell ur friends. 150p/tone. 16 reply HL 4info



## División en set de entrenamiento y prueba

In [190]:
X_train, X_test, y_train, y_test = train_test_split(data.v2, data.v1, train_size = .75, random_state = 1)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(4179,) (4179,) (1393,) (1393,)


## Preprocesamiento

In [191]:
def preprocess(text):
    words = [word.lower() for word in word_tokenize(text)]
    return " ".join(stemmer.stem(word) for word in words if len(word) > 2 and word not in cached_stopwords)

In [192]:
print(f'Original:\t{X_train.iloc[0]}')
print(f'Procesado:\t{preprocess(X_train.iloc[0])}')

Original:	Height of Confidence: All the Aeronautics professors wer calld &amp; they wer askd 2 sit in an aeroplane. Aftr they sat they wer told dat the plane ws made by their students. Dey all hurried out of d plane.. Bt only 1 didnt move... He said:\if it is made by my students
Procesado:	height confid aeronaut professor wer calld amp wer askd sit aeroplan aftr sat wer told dat plane made student dey hurri plane.. didnt move ... said \if made student


In [193]:
# Es exactamente lo mismo que mi preprocesamiento, pero me regresa la lista de 
# palabras en lugar de la cadena unida por espacios
def tokenize(text):
    words = [word.lower() for word in word_tokenize(text)]
    return [stemmer.stem(word) for word in words if len(word) > 2 and word not in cached_stopwords]

# Vectorización

In [194]:
vect = TfidfVectorizer(tokenizer=tokenize, lowercase = False)
X_train_vect = vect.fit_transform(X_train)
X_train_vect

<4179x6824 sparse matrix of type '<class 'numpy.float64'>'
	with 33736 stored elements in Compressed Sparse Row format>

In [195]:
pd.DataFrame(X_train_vect.todense(), index = X_train.index, columns = vect.vocabulary_)

,height,confid,aeronaut,professor,wer,calld,amp,askd,sit,aeroplan,aftr,sat,told,dat,plane,made,student,dey,hurri,plane..,didnt,move,...,said,\if,2/2,146tf150p,wen,lovabl,bcum,angri,wid,dnt,take,seriously..,coz,childish,true,way,show,...,n=qjkgighjjgcbl,tens,player.whi,rt-king,ringtoneking.co.uk,08701237397,redeem,www.ringtoneking.co.uk,accentur,versu,ryder,unsold.now,jide,5.im,dread,thou,pressi,69855,stopbcm,skint,bevies.waz,othr,spoon,watchng,planet,comfey,shouting..,forfeit,highest,maximum,83383,yes.i,jabo,seek,raglan,closebi,pdate_now,call2optout/,yhl,upstair
710,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2711,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3155,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3748,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [196]:
vect = TfidfVectorizer(tokenizer=tokenize, lowercase = False, max_df=.995, min_df=.005)
X_train_vect = vect.fit_transform(X_train)
X_train_vect

<4179x305 sparse matrix of type '<class 'numpy.float64'>'
	with 17232 stored elements in Compressed Sparse Row format>

In [197]:
pd.DataFrame(X_train_vect.todense(), index = X_train.index, columns = vect.vocabulary_)

,amp,told,dat,didnt,...,said,take,way,show,care,luv,nice,day,long,time,rememb,today,dear,voucher,claim,1st,class,use,holiday,call,book,thing,back,home,littl,bore,alreadi,haha,well,get,shit,thank,love,bad,dont,...,lot,feel,msg,reach,dinner,select,problem,birthday,haf,sleep,rington,rate,heart,shop,fine,wake,hous,receiv,enjoy,life,name,hear,lar,mean,hello,bed,collect,di,decid,await,look,may,per,goe,pain,2nd,town,car,forgot,account
710,0.0,0.0,0.0,0.227892,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.410017,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.428629,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3740,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2711,0.0,0.0,0.0,0.211452,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.354897,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3155,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.398155,0.428579,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3748,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.503037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.251518,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.218327,0.499914,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5192,0.0,0.0,0.0,0.573250,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.294472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3980,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
235,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [198]:
X_test_vect = vect.transform(X_test)
X_test_vect

<1393x305 sparse matrix of type '<class 'numpy.float64'>'
	with 5549 stored elements in Compressed Sparse Row format>

## ¿Qué es que sea "sparse" (escaso)?

Para conocer la proporción de elementos que son cero, la siguiente línea de código hace la siguiente operación. 

\begin{align}
\frac{\text{"no. elementos"} - \text{"no. elementos distintos a cero"}}{\text{"no. elementos"}} = \frac{\text{"no. elementos iguales a cero"}}{\text{"no. elementos"}}
\end{align}

In [199]:
(np.multiply(*X_train_vect.shape) - np.count_nonzero(X_train_vect.todense())) / np.multiply(*X_train_vect.shape)

0.9864804114248056

In [200]:
1 - 16024/(3900*307)

0.9866165539129709

# Regresión Logística

## Regresión logística sin regularización

Para $x_i \in \mathbb{R}^p$ y $y_i \in  [0, 1]$ . Los coeficientes de la regresión logística, $\beta = (\beta_0, \beta_1, \dots, \beta_p)$, son tales que se minimiza la función de pérdida

$$\beta = argmin_\beta\frac{1}{m}\sum_{i = 0}^{m}L(\beta, x_i, y_i)$$

donde L es la función de pérdida para cada caso, 

$$L(\beta, x_i, y_i) =  -y_ilog\left(S(\beta, x_i)\right) -(1-y_i)log\left(S(\beta, x_i)\right) $$

y $S$ es la función sigmoide, 

$$S(\beta, x_i) = \frac{1}{1+e^{-\beta^Tx_i}}$$

In [201]:
logistic = LogisticRegression(penalty = 'none', max_iter = 10000)
logistic_train_pred = logistic.fit(X_train_vect, y_train).predict(X_train_vect)
print(classification_report(y_train, logistic_train_pred))

              precision    recall  f1-score   support

         ham       0.99      1.00      1.00      3612
        spam       0.99      0.95      0.97       567

    accuracy                           0.99      4179
   macro avg       0.99      0.98      0.98      4179
weighted avg       0.99      0.99      0.99      4179



In [202]:
logistic_test_pred = logistic.predict(X_test_vect)
print(classification_report(y_test, logistic_test_pred))

              precision    recall  f1-score   support

         ham       0.98      0.97      0.98      1213
        spam       0.82      0.89      0.86       180

    accuracy                           0.96      1393
   macro avg       0.90      0.93      0.92      1393
weighted avg       0.96      0.96      0.96      1393



## Regresión logística con regularización L2 (Ridge)

Con regularización l2, la función a minimizar es

$$\beta = argmin_\beta\frac{1}{m}\sum_{i = 0}^{m}L(\beta, x_i, y_i) \text{    sujeto a    } \|\beta\|_2^2 < \epsilon$$

Su Lagrangiano es,

$$argmin_\beta\frac{1}{m}\sum_{i = 0}^{m}L(\beta, x_i, y_i) + \lambda \|\beta\|_2^2$$


In [279]:
logistic_l2 = LogisticRegression(C = 5)
logistic_l2_train_pred = logistic_l2.fit(X_train_vect, y_train).predict(X_train_vect)
print(classification_report(y_train, logistic_l2_train_pred))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      3612
        spam       0.98      0.87      0.92       567

    accuracy                           0.98      4179
   macro avg       0.98      0.93      0.96      4179
weighted avg       0.98      0.98      0.98      4179



In [280]:
logistic_l2_test_pred = logistic_l2.predict(X_test_vect)
print(classification_report(y_test, logistic_l2_test_pred))

              precision    recall  f1-score   support

         ham       0.98      1.00      0.99      1213
        spam       0.96      0.86      0.91       180

    accuracy                           0.98      1393
   macro avg       0.97      0.93      0.95      1393
weighted avg       0.98      0.98      0.98      1393



## Coeficientes

In [281]:
coefs = pd.Series(logistic_l2.coef_[0], index = vect.vocabulary_).sort_values().reset_index(name = 'n')
px.bar(coefs, 'index', 'n')

In [282]:
word_regex = '[^a-z]rington.*'
for name, (label, txt) in data[data.v2.str.contains(word_regex, case = False)].sort_values("v1").sample(5, replace = True).iterrows():
    print(label, txt, end = '\n\n', sep = ': ')

spam: FREE RINGTONE text FIRST to 87131 for a poly or text GET to 87131 for a true tone! Help? 0845 2814032 16 after 1st free, tones are 3x150pw to end txt stop

spam: Thanks for your ringtone order, ref number R836. Your mobile will be charged 4.50. Should your tone not arrive please call customer services on 09065069154

spam: RECPT 1/3. You have ordered a Ringtone. Your order is being processed...

spam: Free Top ringtone -sub to weekly ringtone-get 1st week free-send SUBPOLY to 81618-?3 per week-stop sms-08718727870

spam: RECPT 1/3. You have ordered a Ringtone. Your order is being processed...



In [283]:
data[data.v2.str.contains(word_regex, case = False)].v1.value_counts() / data.v1.value_counts() 

ham         NaN
spam    0.05087
Name: v1, dtype: float64

# Razón de momios (odds ratio)

En la regresión logística, tomando una variable y poniendo las demás variables fijas, el coeficiente de esta variable, llamemoslo $\beta_i$, nos indica que un cambio en una unidad (para este caso, que la palabra esté, 1, o no, 0) resulta en un cambio $\beta_i$ en las *log odds*. Para convertirlo a la razón de momios, tenemos que exponenciar este coeficiente, esto es, $e^{\beta_i} = OR$. Y la interpretación, para este caso sería la siguiente: *Las posibilidades de que el mensaje de texto sea spam, dado que la palabra x está, son OR veces las posibilidades que si no estuviera*

Veamos un ejemplo considerando la palabra *rington* en la regresión logística con regularización.

In [284]:
np.exp(coefs[coefs['index'] == 'rington'].n)
# 1996.68

304    1996.683887
Name: n, dtype: float64

Esto quiere decir que, si la palabra rington apareciera en el mensaje de texto, las posibilidades de ser clasificado como spam aumentan casi en un 2000%.

In [285]:
texto = 'Hello this is my new phone and my awesome ringtone'
texto_vect = vect.transform([texto])
texto_vect

<1x305 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>

In [286]:
logistic_l2.predict(texto_vect)

array(['spam'], dtype=object)

## Sobre el parámetro de regularización

In [ ]:
# logistic_l2_2 = LogisticRegression(C = 1).fit(X_train_vect, y_train)
# logistic_l2_3 = LogisticRegression(C = 100).fit(X_train_vect, y_train)
# logistic_l2_4 = LogisticRegression(C = .1).fit(X_train_vect, y_train)

In [ ]:
# coefs = pd.DataFrame([logistic.coef_[0], logistic_l2.coef_[0], logistic_l2_2.coef_[0], logistic_l2_3.coef_[0], logistic_l2_4.coef_[0]]).T
# coefs.columns = ['sin_regularizacion', 'c_1', 'c_10', 'c_100', 'c_.1']
# coefs['name'] = vect.vocabulary_
# # coefs.sort_values('c_1', inplace=True)
# coefs


In [ ]:
# coefs.describe()

In [ ]:
# plot_data = coefs.melt(id_vars = 'name')
# plot_data

In [ ]:
# plot_data[plot_data.name == "hey"]

In [ ]:
# px.bar(plot_data, 'name', 'value', color = 'variable', barmode = 'group')